### Import Libraries

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Highlight the detected shape of the template on the image

In [2]:
def highlight_shape(img, temp_img, position):
    result = img.copy()
    result[position[1]: position[1] + temp_img.shape[0],
           position[0]: position[0] + temp_img.shape[1]][temp_img != 0] = [255, 0, 0]
    return result

### Highlight a rectangle shape on the image

In [3]:
def draw_rectangle(img, start, shape, color=[255, 0, 0], thickness=1):
    end = start + shape
    return cv2.rectangle(img, tuple(start), tuple(end), color, thickness)

### Divide template into k segments

In [4]:
def get_segments(template, k):
    def neighbours(pix):
        x = pix[0]
        for y in [pix[1]-1, pix[1]+1]:
            yield x, y

        y = pix[1]
        for x in [pix[0]-1, pix[0]+1]:
            yield x, y

        for x in [pix[0]-1, pix[0]+1]:
            for y in [pix[1]-1, pix[1]+1]:
                yield x, y

    segment_length = int(np.ceil(template.sum() / 255 / k))
    xy_pixels = np.nonzero(template)
    pixels = list(zip(xy_pixels[0], xy_pixels[1]))
    
    segments = []
    segment = np.zeros_like(template)
    queue = []

    old_pixel = pixels[330]  # Will deal with this later
    segment[old_pixel[0], old_pixel[1]] = 255
    pixels.remove(old_pixel)
    
    for i in range(1, int(template.sum()/255)):
        for x, y in neighbours(old_pixel):
            if (x, y) in pixels and (x, y) not in queue:
                queue.append((x,y))

        pixel = queue.pop(0)
        segment[pixel[0], pixel[1]] = 255
        pixels.remove(pixel)
        old_pixel = pixel

        if (i+1) % segment_length == 0:
            segments.append(segment)
            segment = np.zeros_like(template)
    else:
        if segment.any():
            segments.append(segment)
            
    return segments

### Reject Outliers of a Dataset

In [5]:
def reject_outliers(data, m=2, axis=0):
    d = np.abs(data - np.median(top_left_idx, axis=axis))
    mdev = np.median(d, axis=axis)
    s = d
    for i in range(len(s[axis])):
        s[:, i] = d[:, i]/mdev[0] if mdev[0] else 0.
    return data[np.all(s<m, axis=-1)]    

### Calculate the Mean of a Dataset

In [6]:
def get_mean(data, axis=0):
    return np.mean(data, axis=axis)

### Hyperparameters

In [7]:
blur_size = 5
blur_sigma = 1
canny_low_th = 70
canny_up_th = 150

n_segments = 10
n_max = 300
alpha = 0.8
memory_size = 10

### Read template image and video

In [8]:
temp = cv2.imread('hand_template.bmp', 0)
height, width = temp.shape[::]
cap = cv2.VideoCapture('test2.wmv')
frame_exists, frame = cap.read()

### Initializations

In [9]:
memory = np.zeros((memory_size, 2))
counter = 0

segments = get_segments(temp, n_segments)

### Main loop

In [10]:
# For each frame
while frame_exists:    
    cm = []
    max_ind = []
    
    # Preprocessing
    top, bottom, left, right = [height // 4] * 2 + [width // 4] * 2
    frame_padded = cv2.copyMakeBorder(frame, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (blur_size, blur_size), blur_sigma)
    edges = cv2.Canny(gray, canny_low_th, canny_up_th)
    edges = cv2.copyMakeBorder(edges, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    
    # Calculate distance transform
    dist = cv2.distanceTransform(255 - edges, cv2.DIST_L2, 3)
    cv2.normalize(dist, dist, 0, 255, cv2.NORM_MINMAX)
    dist = dist.astype(np.uint8)
    
    # Apply chamfer matching and save the indeces of the top (n_max) values for each segment
    for seg in segments:
        cm.append((cv2.matchTemplate(255 - dist, seg, cv2.TM_CCORR) / seg.sum()))
        ind = np.unravel_index((-1*cm[-1]).argsort(axis=None), cm[-1].shape)
        max_ind.append(np.array([ind[0][:n_max], ind[1][:n_max]]))
    
    # Get the indeces that minimize a cost function that depends on the matching values and the distance between segments
    cost_old = alpha * (1 - (cm[0][tuple(max_ind[0])] / 255))
    idx_tracker = [[x] for x in range(n_max)]
    for i in range(1, n_segments):
        cost = cost_old[:, None] + alpha * (1 - (cm[i][tuple(max_ind[i])] / 255)) + \
               (1 - alpha) * np.exp(2 * (np.linalg.norm(max_ind[i] - max_ind[i - 1], axis=0)))
        idx = np.unravel_index((cost).argsort(axis=None), cost.shape)
        idx = (idx[0][:n_max], idx[1][:n_max])
        cost_old = cost[idx]
        idx_tracker1 = idx_tracker.copy()
        idx_tracker = [idx_tracker1[x] + [y] for x, y in list(zip(idx[0], idx[1]))]
    tracked_idx = idx_tracker[0]
    top_left_idx = np.array([np.array(max_ind)[x, :, tracked_idx[x]] for x in range(n_segments)])
    
    # Highlight each segment at its maximum value index
    hl_img = frame_padded
    for i in range(n_segments):
        hl_img = highlight_shape(hl_img, segments[i], (top_left_idx[i, 1], top_left_idx[i, 0]))
    
    # Reject indeces outliers then calculate the mean
    top_left_idx_filtered = reject_outliers(top_left_idx)
    top_left = get_mean(top_left_idx_filtered).astype(np.int64)
    
    # Calculate the mean of the indeces for a number of consecutive frames (for smoothing)
    if(np.all(top_left >= 0)):
        memory[counter % memory_size] = top_left
        if(counter >= (memory_size - 1)):
            top_left = get_mean(memory).astype(np.int64)
    
    # Highlight a rectangular shape at the calculated mean index
    try:
        rect_hl_img = draw_rectangle(frame_padded, top_left[::-1], temp.shape[::-1])
    except TypeError:
        rect_hl_img = frame_padded
    
    # Show results
    cv2.imshow('dt', dist)
    cv2.imshow('Edges', edges)
    cv2.imshow('Tracking', hl_img)
    cv2.imshow('Tracking2', rect_hl_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # Get next frame
    frame_exists, frame = cap.read()
    counter += 1

cap.release()
cv2.destroyAllWindows()

C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\al_kosa\anaconda3\lib\site-packages\numpy\core\_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
